<a href="https://colab.research.google.com/github/yuri-feital/trabalho-spark/blob/main/Projeto_Spark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar Arquivo do Drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
os.chdir("/content/drive/MyDrive/MIT_Infnet/Aula_Hadoop/Projeto")

Instalar o pyspark

In [10]:
!pip install pyspark

Importar SparkSession ao invés de SparkContext diretamente para trabalhar com DataFrames

In [11]:
from pyspark.sql import SparkSession

Criar a SparkSession

In [12]:
spark = SparkSession.builder \
    .appName("Auxilio Emergencial") \
    .master("local") \
    .getOrCreate()

Verificar o nome da aplicação (opcional)

In [13]:
print(spark.sparkContext.appName)

Auxilio Emergencial


Ler o arquivo CSV usando DataFrame API (usando header=True)

In [ ]:
municipio_df = spark.read.csv("POP2022_Municipios_20230622.csv",sep=';', header=True, inferSchema=True)
auxilio_df = spark.read.csv("auxilio_emergencial (1).csv",sep=';', header=True, inferSchema=True)
joined_df = auxilio_df.join(municipio_df, auxilio_df["uf"] == municipio_df["uf"])

Mostrar as primeiras 5 linhas do DataFrame

In [ ]:
municipio_df.show(5)
auxilio_df.show(5)

Converter DataFrame para RDD

In [ ]:
municipio_rdd = municipio_df.rdd
auxilio_rdd = auxilio_df.rdd

Mostrar as primeiras linhas do RDD

In [ ]:
municipio_rdd.take(5)
auxilio_rdd.take(5)

In [ ]:
populacao_por_uf = municipio_df.groupBy("uf").agg({" POPULAÇÃO ": "sum"}).withColumnRenamed("sum( POPULAÇÃO )", "total_pessoa")
populacao_por_uf.show(n=populacao_por_uf.count(), truncate=False)